In [1]:
pip_source = "hi-ml-multimodal"

In [3]:
%pip install {pip_source}

^C
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.3.1 requires opencv-python-headless>=4.1.1, which is not installed.
torchaudio 2.1.1+cu118 requires torch==2.1.1+cu118, but you have torch 1.9.0 which is incompatible.


Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ---------------------------------------- 0.0/84.4 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/84.4 kB ? eta -:--:--
     --------- ---------------------------- 20.5/84.4 kB 330.3 kB/s eta 0:00:01
     --------- ---------------------------- 20.5/84.4 kB 330.3 kB/s eta 0:00:01
     --------- ---------------------------- 20.5/84.4 kB 330.3 kB/s eta 0:00:01
     --------- ---------------------------- 20.5/84.4 kB 330.3 kB/s eta 0:00:01
     --------- ---------------------------- 20.5/84.4 kB 330.3 kB/s eta 0:00:01
     --------- ---------------------------- 20.5/84.4 kB 330.3 kB/s eta 0:00:01
     --------- ---------------------------- 20.5/84.4 kB 330.3 kB/s eta 0:00:01
     --------- ---------------------------- 20.5/84.4 kB 330.3 kB/s eta 0:00:01
     --------- ---------------------------- 20.5/84.4 kB 330.3 kB/s eta 0:00:01
     --------- ---------------------------- 20.5/84.4 kB 330.3 kB/s eta 0:00

In [4]:
from typing import List
from typing import Tuple

import tempfile
from pathlib import Path

import torch
from IPython.display import display
from IPython.display import Markdown

from health_multimodal.common.visualization import plot_phrase_grounding_similarity_map
from health_multimodal.text import get_bert_inference
from health_multimodal.text.utils import BertEncoderType
from health_multimodal.image import get_image_inference
from health_multimodal.image.utils import ImageModelType
from health_multimodal.vlp import ImageTextInferenceEngine

d:\anaconda\envs\dcr\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
text_inference = get_bert_inference(BertEncoderType.BIOVIL_T_BERT)
image_inference = get_image_inference(ImageModelType.BIOVIL_T)

ValueError: Connection error, and we cannot find the requested files in the cached path. Please try again or make sure your Internet connection is on.

In [ ]:
image_text_inference = ImageTextInferenceEngine(
    image_inference_engine=image_inference,
    text_inference_engine=text_inference,
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_text_inference.to(device)

In [ ]:
TypeBox = Tuple[float, float, float, float]

def plot_phrase_grounding(image_path: Path, text_prompt: str, bboxes: List[TypeBox]) -> None:
    similarity_map = image_text_inference.get_similarity_map_from_raw_data(
        image_path=image_path,
        query_text=text_prompt,
        interpolation="bilinear",
    )
    plot_phrase_grounding_similarity_map(
        image_path=image_path,
        similarity_map=similarity_map,
        bboxes=bboxes
    )

def plot_phrase_grounding_from_url(image_url: str, text_prompt: str, bboxes: List[TypeBox]) -> None:
    image_path = Path(tempfile.tempdir, "downloaded_chest_xray.jpg")
    !curl -s -L -o {image_path} {image_url}
    plot_phrase_grounding(image_path, text_prompt, bboxes)

In [ ]:
image_url = "https://openi.nlm.nih.gov/imgs/512/242/1445/CXR1445_IM-0287-4004.png"
text_prompt = "Left basilar consolidation seen"
# Ground-truth bounding box annotation(s) for the input text prompt
bboxes = [
    (306, 168, 124, 101),
]

text = (
    'The ground-truth bounding box annotation for the phrase'
    f' *{text_prompt}* is shown in the middle figure (in black).'
)

display(Markdown(text))
plot_phrase_grounding_from_url(image_url, text_prompt, bboxes)